# Power Plant

In [1]:
# Import the necessary libraries.

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

# Display all the columns in a dataset

pd.set_option("display.max_columns", None)

import catboost
import sklearn

In [2]:
# load the power dataset 

power = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\53 season Multinomial Classifier 05 - Aug - 2021\Power_Plant_S1.csv", header=0) 
power.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [3]:
# Display the dataset information

power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [4]:
# Identify the independent and Target (dependent) variables

IndepVar = []
for col in power.columns:
    if col != 'PE':
        IndepVar.append(col)

TargetVar = 'PE'

x = power[IndepVar]
y = power[TargetVar]

In [5]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7654, 4), (1914, 4), (7654,), (1914,))

In [6]:
# Train the algorithm and build the model with train dataset

from catboost import CatBoostRegressor
from catboost.utils import eval_metric

modelCBR = CatBoostRegressor(iterations=100, verbose=10, loss_function='RMSE')
modelCBR.fit(x_train, y_train, eval_set=(x_test, y_test)) 
modelCBR.set_feature_names(x)

Learning rate set to 0.284146
0:	learn: 12.9473862	test: 12.8954375	best: 12.8954375 (0)	total: 154ms	remaining: 15.3s
10:	learn: 4.2114581	test: 4.2153673	best: 4.2153673 (10)	total: 211ms	remaining: 1.71s
20:	learn: 3.9776106	test: 4.0138142	best: 4.0138142 (20)	total: 236ms	remaining: 889ms
30:	learn: 3.8256950	test: 3.8795081	best: 3.8795081 (30)	total: 260ms	remaining: 579ms
40:	learn: 3.6900301	test: 3.7796229	best: 3.7796229 (40)	total: 285ms	remaining: 410ms
50:	learn: 3.5644000	test: 3.6718692	best: 3.6718692 (50)	total: 309ms	remaining: 297ms
60:	learn: 3.4770413	test: 3.6129131	best: 3.6129131 (60)	total: 333ms	remaining: 213ms
70:	learn: 3.3868830	test: 3.5539090	best: 3.5539090 (70)	total: 358ms	remaining: 146ms
80:	learn: 3.3167089	test: 3.5063338	best: 3.5063338 (80)	total: 382ms	remaining: 89.6ms
90:	learn: 3.2413084	test: 3.4508127	best: 3.4508127 (90)	total: 409ms	remaining: 40.4ms
99:	learn: 3.1845162	test: 3.4191891	best: 3.4191891 (99)	total: 431ms	remaining: 0us



In [7]:
# Predict the model

y_pred = modelCBR.predict(x_test)

# Evaluation metrics for CatBoost Regressor 

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.56
Mean Squared Error (MSE): 11.691
Root Mean Squared Error (RMSE): 3.419
Mean Absolute Percentage Error (MAPE): 0.6 %
R2_score: 0.959695
Adj R Square:  0.959678


In [8]:
# Important Attributes & Methods of CatBoostRegressor

print("Best Score                : ", modelCBR.best_score_)
#print("\nList of Target Classses : ", modelCBR.classes_)
print("\nData Feature Names      : ", modelCBR.feature_names_)
print("\nFeature Importance      : ", modelCBR.feature_importances_)
print("\nLearning Rate           : ", modelCBR.learning_rate_)
print("\nRandom Seed             : ", modelCBR.random_seed_)
print("\nNumber of Trees         : ", modelCBR.tree_count_)
print("\nNumber of Features      : ", modelCBR.n_features_in_)

Best Score                :  {'learn': {'RMSE': 3.184516204101079}, 'validation': {'RMSE': 3.4191890921817625}}

Data Feature Names      :  ['AT', 'V', 'AP', 'RH']

Feature Importance      :  [63.20423683 25.57295059  7.08022102  4.14259156]

Learning Rate           :  0.2841460108757019

Random Seed             :  0

Number of Trees         :  100

Number of Features      :  4


In [9]:
# CatBoost Methods:

print("\nBest Score                  : ", modelCBR.get_best_score())
print("\nParameters Passed When Creating Model : ", modelCBR.get_params())
print("\nAll Model Parameters                : ", modelCBR.get_all_params())
#print("\nCategorical Feature Indices : ", modelCBR.get_cat_feature_indices())
print("\nFeature Importances        : ", modelCBR.get_feature_importance())
print("\nLeaf Values Shape   : ", modelCBR.get_leaf_values().shape)
print("\nLeaf Values         : ", modelCBR.get_leaf_values()[:10])
print("\nLeaft Weights Shape : ", modelCBR.get_leaf_weights().shape)
print("\nLeaft Weights       : ", modelCBR.get_leaf_weights()[:10])


Best Score                  :  {'learn': {'RMSE': 3.184516204101079}, 'validation': {'RMSE': 3.4191890921817625}}

Parameters Passed When Creating Model :  {'iterations': 100, 'loss_function': 'RMSE', 'verbose': 10}

All Model Parameters                :  {'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 100, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': True, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovem

In [10]:
leaf_indices = modelCBR.calc_leaf_indexes(x_train)

print("Leaf Indices Size : ", leaf_indices.shape)

leaf_indices[:2]

Leaf Indices Size :  (7654, 100)


array([[11, 23, 13, 55,  7, 59, 10, 10, 21, 18, 55,  6,  9, 55, 26, 27,
        61, 28, 37, 27,  4, 14, 55,  0, 42, 60, 61, 12, 21, 11, 31, 52,
        32, 59, 49, 62,  0,  0,  0, 60, 62, 40, 17, 46,  2, 27, 11,  7,
        33, 40, 16, 63, 61, 31, 24,  1, 48, 34, 55,  1, 27,  0,  4, 32,
        55,  1, 59, 32, 46, 40, 31, 60,  0, 30, 17, 63,  3, 47, 59, 48,
         4,  0,  1, 30,  8, 31, 63, 19,  0, 57, 31, 48, 27, 47,  0, 59,
        55, 19, 33, 59],
       [24, 52, 16, 20, 16, 10,  6, 10, 36,  2, 36, 36, 33, 10, 30, 51,
         0, 46, 37, 36, 36,  8,  8,  8, 22,  4, 13,  2, 21, 11, 49,  8,
        12, 29,  7,  2, 16, 32,  0, 23,  0, 22, 35, 22, 10,  3, 16, 48,
         7, 25, 24,  3, 61, 61, 24, 17, 12, 18, 24,  1, 31,  0,  0, 44,
        31,  1, 47, 40, 22, 40, 33, 10, 55, 34,  7, 63, 16, 17,  3, 48,
         4,  0,  7, 34,  8, 32, 35, 51, 16, 15,  1, 51, 63, 16, 34, 45,
        31, 51, 57, 20]], dtype=uint32)

In [11]:
# Build the CatBoost with Pool function

from catboost import Pool

# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

train_data = catboost.Pool(x_train, y_train)
test_data = catboost.Pool(x_test, y_test)

# Build the model

from catboost import CatBoostRegressor
from catboost import CatBoost
from catboost.utils import eval_metric

modelCBR = catboost.CatBoostRegressor(loss_function='RMSE')

params = {'iterations': [100, 150, 200],
          'learning_rate': [0.03, 0.1],
          'depth': [2, 4, 6, 8],
          'l2_leaf_reg': [0.2, 0.5, 1, 3]}

modelCBR.grid_search(params, train_data)

0:	learn: 441.0567038	test: 441.3533869	best: 441.3533869 (0)	total: 5.35ms	remaining: 529ms
1:	learn: 427.8442074	test: 428.1426105	best: 428.1426105 (1)	total: 8.7ms	remaining: 426ms
2:	learn: 415.0156087	test: 415.3135560	best: 415.3135560 (2)	total: 10.6ms	remaining: 344ms
3:	learn: 402.5833472	test: 402.8717144	best: 402.8717144 (3)	total: 12.1ms	remaining: 291ms
4:	learn: 390.5155435	test: 390.8045152	best: 390.8045152 (4)	total: 13.8ms	remaining: 263ms
5:	learn: 378.8067946	test: 379.0900322	best: 379.0900322 (5)	total: 16ms	remaining: 250ms
6:	learn: 367.4553101	test: 367.7372578	best: 367.7372578 (6)	total: 18.3ms	remaining: 243ms
7:	learn: 356.4440801	test: 356.7292176	best: 356.7292176 (7)	total: 19.7ms	remaining: 227ms
8:	learn: 345.7571679	test: 346.0301457	best: 346.0301457 (8)	total: 20.8ms	remaining: 211ms
9:	learn: 335.3907848	test: 335.6441164	best: 335.6441164 (9)	total: 22ms	remaining: 198ms
10:	learn: 325.3448531	test: 325.5999672	best: 325.5999672 (10)	total: 23.5

57:	learn: 4.7078390	test: 4.6782520	best: 4.6782520 (57)	total: 62.1ms	remaining: 45ms
58:	learn: 4.6700210	test: 4.6450198	best: 4.6450198 (58)	total: 63.3ms	remaining: 44ms
59:	learn: 4.6448818	test: 4.6267250	best: 4.6267250 (59)	total: 64.4ms	remaining: 42.9ms
60:	learn: 4.6208142	test: 4.6070744	best: 4.6070744 (60)	total: 65.5ms	remaining: 41.9ms
61:	learn: 4.5976002	test: 4.5862990	best: 4.5862990 (61)	total: 66.7ms	remaining: 40.9ms
62:	learn: 4.5782878	test: 4.5671783	best: 4.5671783 (62)	total: 67.8ms	remaining: 39.8ms
63:	learn: 4.5605023	test: 4.5516204	best: 4.5516204 (63)	total: 69.3ms	remaining: 39ms
64:	learn: 4.5455915	test: 4.5368556	best: 4.5368556 (64)	total: 70.4ms	remaining: 37.9ms
65:	learn: 4.5308251	test: 4.5238782	best: 4.5238782 (65)	total: 71.5ms	remaining: 36.8ms
66:	learn: 4.5177356	test: 4.5123937	best: 4.5123937 (66)	total: 72.5ms	remaining: 35.7ms
67:	learn: 4.5065222	test: 4.5027653	best: 4.5027653 (67)	total: 73.7ms	remaining: 34.7ms
68:	learn: 4.498

6:	learn: 217.7153928	test: 217.9353571	best: 217.9353571 (6)	total: 7.89ms	remaining: 105ms
7:	learn: 195.9930125	test: 196.2197925	best: 196.2197925 (7)	total: 9.31ms	remaining: 107ms
8:	learn: 176.4233278	test: 176.6185544	best: 176.6185544 (8)	total: 11ms	remaining: 111ms
9:	learn: 158.8099056	test: 158.9595499	best: 158.9595499 (9)	total: 12.1ms	remaining: 109ms
10:	learn: 142.9619496	test: 143.0865587	best: 143.0865587 (10)	total: 13.3ms	remaining: 108ms
11:	learn: 128.7354410	test: 128.8659441	best: 128.8659441 (11)	total: 14.3ms	remaining: 105ms
12:	learn: 115.9053939	test: 116.0030505	best: 116.0030505 (12)	total: 15.4ms	remaining: 103ms
13:	learn: 104.3546719	test: 104.4338602	best: 104.4338602 (13)	total: 16.5ms	remaining: 101ms
14:	learn: 93.9648128	test: 94.0295410	best: 94.0295410 (14)	total: 17.6ms	remaining: 99.6ms
15:	learn: 84.6102330	test: 84.6690991	best: 84.6690991 (15)	total: 18.6ms	remaining: 97.9ms
16:	learn: 76.2099672	test: 76.2690747	best: 76.2690747 (16)	tot

58:	learn: 75.8921608	test: 76.0236956	best: 76.0236956 (58)	total: 64.5ms	remaining: 44.8ms
59:	learn: 73.6317776	test: 73.7557030	best: 73.7557030 (59)	total: 65.6ms	remaining: 43.7ms
60:	learn: 71.4451980	test: 71.5704179	best: 71.5704179 (60)	total: 66.6ms	remaining: 42.6ms
61:	learn: 69.3305359	test: 69.4578277	best: 69.4578277 (61)	total: 67.7ms	remaining: 41.5ms
62:	learn: 67.2776943	test: 67.4039616	best: 67.4039616 (62)	total: 68.7ms	remaining: 40.4ms
63:	learn: 65.2771381	test: 65.3946977	best: 65.3946977 (63)	total: 69.8ms	remaining: 39.2ms
64:	learn: 63.3371826	test: 63.4475900	best: 63.4475900 (64)	total: 70.9ms	remaining: 38.2ms
65:	learn: 61.4553951	test: 61.5592775	best: 61.5592775 (65)	total: 71.9ms	remaining: 37.1ms
66:	learn: 59.6362251	test: 59.7384171	best: 59.7384171 (66)	total: 73ms	remaining: 35.9ms
67:	learn: 57.8655967	test: 57.9590753	best: 57.9590753 (67)	total: 74.1ms	remaining: 34.9ms
68:	learn: 56.1589659	test: 56.2516026	best: 56.2516026 (68)	total: 75.2

22:	learn: 226.1187107	test: 226.3088962	best: 226.3088962 (22)	total: 23.5ms	remaining: 78.6ms
23:	learn: 219.3635423	test: 219.5581060	best: 219.5581060 (23)	total: 24.6ms	remaining: 77.8ms
24:	learn: 212.8069396	test: 212.9970454	best: 212.9970454 (24)	total: 25.6ms	remaining: 76.7ms
25:	learn: 206.4491988	test: 206.6352572	best: 206.6352572 (25)	total: 26.7ms	remaining: 76.1ms
26:	learn: 200.2763430	test: 200.4580189	best: 200.4580189 (26)	total: 27.9ms	remaining: 75.4ms
27:	learn: 194.2924647	test: 194.4726001	best: 194.4726001 (27)	total: 28.9ms	remaining: 74.4ms
28:	learn: 188.4868919	test: 188.6616329	best: 188.6616329 (28)	total: 30ms	remaining: 73.4ms
29:	learn: 182.8518606	test: 183.0106695	best: 183.0106695 (29)	total: 31ms	remaining: 72.2ms
30:	learn: 177.3855644	test: 177.5439007	best: 177.5439007 (30)	total: 32ms	remaining: 71.2ms
31:	learn: 172.0922701	test: 172.2496568	best: 172.2496568 (31)	total: 33.4ms	remaining: 71ms
32:	learn: 166.9506464	test: 167.1090965	best: 1

...

110:	learn: 17.3331054	test: 17.3766587	best: 17.3766587 (110)	total: 457ms	remaining: 366ms
111:	learn: 16.8736703	test: 16.9170994	best: 16.9170994 (111)	total: 461ms	remaining: 362ms
112:	learn: 16.4276188	test: 16.4744750	best: 16.4744750 (112)	total: 465ms	remaining: 358ms
113:	learn: 15.9956001	test: 16.0427700	best: 16.0427700 (113)	total: 470ms	remaining: 355ms
114:	learn: 15.5766040	test: 15.6266183	best: 15.6266183 (114)	total: 479ms	remaining: 354ms
115:	learn: 15.1689322	test: 15.2225960	best: 15.2225960 (115)	total: 491ms	remaining: 355ms
116:	learn: 14.7789461	test: 14.8344026	best: 14.8344026 (116)	total: 499ms	remaining: 354ms
117:	learn: 14.3990922	test: 14.4570011	best: 14.4570011 (117)	total: 509ms	remaining: 353ms
118:	learn: 14.0338843	test: 14.0918309	best: 14.0918309 (118)	total: 517ms	remaining: 352ms
119:	learn: 13.6790747	test: 13.7399197	best: 13.7399197 (119)	total: 525ms	remaining: 350ms
120:	learn: 13.3331454	test: 13.3964508	best: 13.3964508 (120)	total: 

17:	learn: 70.8757636	test: 70.8364216	best: 70.8364216 (17)	total: 187ms	remaining: 1.89s
18:	learn: 64.0462284	test: 64.0428961	best: 64.0428961 (18)	total: 197ms	remaining: 1.87s
19:	learn: 57.8118848	test: 57.8057523	best: 57.8057523 (19)	total: 205ms	remaining: 1.85s
20:	learn: 52.1641134	test: 52.1521898	best: 52.1521898 (20)	total: 215ms	remaining: 1.83s
21:	learn: 47.0917005	test: 47.0873509	best: 47.0873509 (21)	total: 224ms	remaining: 1.81s
22:	learn: 42.5774182	test: 42.5696189	best: 42.5696189 (22)	total: 232ms	remaining: 1.79s
23:	learn: 38.4836887	test: 38.4785350	best: 38.4785350 (23)	total: 241ms	remaining: 1.77s
24:	learn: 34.8144311	test: 34.7978575	best: 34.7978575 (24)	total: 254ms	remaining: 1.78s
25:	learn: 31.4915638	test: 31.4862575	best: 31.4862575 (25)	total: 263ms	remaining: 1.76s
26:	learn: 28.5185470	test: 28.5131987	best: 28.5131987 (26)	total: 271ms	remaining: 1.74s
27:	learn: 25.8563752	test: 25.8529854	best: 25.8529854 (27)	total: 279ms	remaining: 1.72s

123:	learn: 4.1910263	test: 4.8230029	best: 4.8230029 (123)	total: 960ms	remaining: 588ms
124:	learn: 4.1806229	test: 4.8190968	best: 4.8190968 (124)	total: 964ms	remaining: 578ms
125:	learn: 4.1749367	test: 4.8145421	best: 4.8145421 (125)	total: 968ms	remaining: 568ms
126:	learn: 4.1659839	test: 4.8091307	best: 4.8091307 (126)	total: 972ms	remaining: 559ms
127:	learn: 4.1587024	test: 4.8018426	best: 4.8018426 (127)	total: 976ms	remaining: 549ms
128:	learn: 4.1509456	test: 4.7962714	best: 4.7962714 (128)	total: 980ms	remaining: 539ms
129:	learn: 4.1406400	test: 4.7915507	best: 4.7915507 (129)	total: 986ms	remaining: 531ms
130:	learn: 4.1343002	test: 4.7873001	best: 4.7873001 (130)	total: 991ms	remaining: 522ms
131:	learn: 4.1288692	test: 4.7828291	best: 4.7828291 (131)	total: 995ms	remaining: 513ms
132:	learn: 4.1213799	test: 4.7769845	best: 4.7769845 (132)	total: 999ms	remaining: 503ms
133:	learn: 4.1128357	test: 4.7682103	best: 4.7682103 (133)	total: 1s	remaining: 494ms
134:	learn: 4

160:	learn: 3.3638496	test: 3.9018489	best: 3.9018489 (160)
161:	learn: 3.3582152	test: 3.8977058	best: 3.8977058 (161)
162:	learn: 3.3525890	test: 3.8945462	best: 3.8945462 (162)
163:	learn: 3.3483230	test: 3.8919111	best: 3.8919111 (163)
164:	learn: 3.3435637	test: 3.8904995	best: 3.8904995 (164)
165:	learn: 3.3383986	test: 3.8902009	best: 3.8902009 (165)
166:	learn: 3.3351616	test: 3.8896129	best: 3.8896129 (166)
167:	learn: 3.3292940	test: 3.8876419	best: 3.8876419 (167)
168:	learn: 3.3240461	test: 3.8845931	best: 3.8845931 (168)
169:	learn: 3.3200391	test: 3.8822909	best: 3.8822909 (169)
170:	learn: 3.3154606	test: 3.8798280	best: 3.8798280 (170)
171:	learn: 3.3113681	test: 3.8757927	best: 3.8757927 (171)
172:	learn: 3.3063785	test: 3.8724658	best: 3.8724658 (172)
173:	learn: 3.3020289	test: 3.8713064	best: 3.8713064 (173)
174:	learn: 3.2977632	test: 3.8677803	best: 3.8677803 (174)
175:	learn: 3.2937351	test: 3.8659943	best: 3.8659943 (175)
176:	learn: 3.2901275	test: 3.8654498	be

{'params': {'depth': 6,
  'iterations': 200,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.2},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [12]:
# Predict the model

y_pred = modelCBR.predict(test_data)

# Evaluation metrics for CatBoost Regressor 

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.59
Mean Squared Error (MSE): 11.833
Root Mean Squared Error (RMSE): 3.44
Mean Absolute Percentage Error (MAPE): 0.6 %
R2_score: 0.959204
Adj R Square:  0.959187


In [13]:
print("\nParameters Passed When Creating Model : ", modelCBR.get_params())


Parameters Passed When Creating Model :  {'loss_function': 'RMSE', 'depth': 6, 'iterations': 200, 'learning_rate': 0.1, 'l2_leaf_reg': 0.2}


# Muulti-Regression Algorithm

In [14]:
# Train the algorithm and build the model with train dataset

from sklearn.linear_model import LinearRegression

modelREG = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None, positive=False)  

modelREG.fit(x_train, y_train) 

# Predict the model with test dataset

y_pred = modelREG.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.596
Mean Squared Error (MSE): 20.274
Root Mean Squared Error (RMSE): 4.503
Mean Absolute Percentage Error (MAPE): 0.8 %
R2_score: 0.930105
Adj R Square:  0.930076


# Decision Tree Regressor

In [15]:
# Build the Decision Tree Regressor model

from sklearn.tree import DecisionTreeRegressor

modelDTR = DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None, min_samples_split=2,
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                                 random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                 min_impurity_split=None, ccp_alpha=0.0)

modelDTR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred1 = modelDTR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred1),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred1),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred1)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred1),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred1),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred1),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.004
Mean Squared Error (MSE): 19.355
Root Mean Squared Error (RMSE): 4.399
Mean Absolute Percentage Error (MAPE): 0.66 %
R2_score: 0.933271
Adj R Square:  0.933243


# Random Forest Regressor

In [16]:
# build a random forest regression model with optimized hyper parameters

from sklearn.ensemble import RandomForestRegressor

bostonRFR = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2,
                                  min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                                  max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
                                  bootstrap=True, oob_score=False, n_jobs=None, random_state=None,
                                  verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)

bostonRFR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred2 = bostonRFR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred2),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred2),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred2)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred2),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred2),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred2),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.314
Mean Squared Error (MSE): 10.395
Root Mean Squared Error (RMSE): 3.224
Mean Absolute Percentage Error (MAPE): 0.51 %
R2_score: 0.964161
Adj R Square:  0.964146


# SVR - Linear Kernel

In [17]:
# Build the SVR model

from sklearn.svm import SVR

bostonSVR = SVR(C=1.0, kernel='linear', degree=3, gamma='scale', coef0=0.0, tol=0.001,
                epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=- 1)

bostonSVR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred3 = bostonSVR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred3),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred3),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred3)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred3),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred3),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred3),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.598
Mean Squared Error (MSE): 20.587
Root Mean Squared Error (RMSE): 4.537
Mean Absolute Percentage Error (MAPE): 0.79 %
R2_score: 0.929025
Adj R Square:  0.928995


# KNN Regressor

In [18]:
# Build the algorithm with KNN

from sklearn import neighbors

bostonKNN = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto',
                                          leaf_size=30, p=2, metric='minkowski',
                                          metric_params=None, n_jobs=None)
bostonKNN.fit(x_train, y_train)

# Predict the model with test dataset

y_pred4 = bostonKNN.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred4),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred4),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred4)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred4),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred4),6))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred4),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.884
Mean Squared Error (MSE): 15.383
Root Mean Squared Error (RMSE): 3.922
Mean Absolute Percentage Error (MAPE): 0.64 %
R2_score: 0.946965
Adj R Square:  0.946943
